# Step 1: Project Mining - GitHub

In [1]:
import os
import datetime
import random
import json
try: 
    from github import Github
    import pickle
except ImportError:
    print("Lib github not local available")
    import pip
    cmd = "pip install PyGithub"
    os.system(cmd)
    cmd = "pip install pickle"
    os.system(cmd)
    import pickle
CONFIG_FILES_PATH = "/home/configFiles/GitHubProjects/"

In [2]:
from github import Github

g = Github("maes95","xxxx")

In [ ]:
query="""
    language:java 
    stars:>=500 
    forks:>=300 
    created:<2015-01-01 
    pushed:>2020-01-01
    archived:false
    is:public
"""
repositories=[]

repositories = g.search_repositories(query=query)

In [ ]:
repos= []
count=0

for repo in repositories:
    repos.append(repo)
    count=count+1

In [ ]:
date=str(datetime.datetime.now())

with open('repos_%s.pickle'%date, 'wb') as f:
    pickle.dump(repos, f)

print("Total repos: %d"%len(repos))

In [3]:
file_name = "2020-03-03 16:33:38.327680"
if 'date' in locals(): file_name = date

# 'repos_%s.pickle'%date
with open('repos_%s.pickle'%file_name, 'rb') as f:
    repos = pickle.load(f)

In [4]:
filtered_repos = []
for r in repos:
    
    try:
        repo = g.get_repo(r.full_name)
        contents = repo.get_contents("")
        for content_file in contents:
            if content_file.path in ["build.gradle", "pom.xml", "build.xml"]:
                filtered_repos.append(repo)
                break
    except:
        print("Project %s not longer available"%r.full_name)
        continue
            
print("Total repos: %d"%len(filtered_repos))

Project psaravan/JamsMusicPlayer not longer available
Project eclipse/jgit not longer available
Project eclipse/egit-github not longer available
Total repos: 590


In [5]:
MAX_COMMITS = 10000
MIN_COMMITS = 1000
filtered_repos_2 = []
for repo in filtered_repos:
    commits = repo.get_commits().totalCount
    if commits >= MIN_COMMITS and commits <= MAX_COMMITS:
        filtered_repos_2.append(repo)
print("Total projects %d"%len(filtered_repos_2))

Total projects 302


In [ ]:
#filtered_repos_3 = [r for r in filtered_repos_2 if r.full_name.split("/")[0] != "apache"]
#print("Total projects %d"%len(filtered_repos_3))

In [6]:
filtered_repos_3 = []
for repo in filtered_repos_2:
    contents = repo.get_contents("")
    isAndroid = False
    for file in contents:
        if file.path == "build.gradle":
            isAndroid = 'com.android.tools.build' in str(repo.get_contents("build.gradle").decoded_content)
            break
    if not isAndroid: filtered_repos_3.append(repo)
print("Total projects %d"%len(filtered_repos_3))

Total projects 255


In [7]:
# Select 80 random projects
sampling = random.choices(filtered_repos_3, k=80)

In [8]:
sampling

[Repository(full_name="undertow-io/undertow"),
 Repository(full_name="appium/java-client"),
 Repository(full_name="mock-server/mockserver"),
 Repository(full_name="mock-server/mockserver"),
 Repository(full_name="jindrapetrik/jpexs-decompiler"),
 Repository(full_name="plutext/docx4j"),
 Repository(full_name="flyway/flyway"),
 Repository(full_name="kiegroup/optaplanner"),
 Repository(full_name="apache/drill"),
 Repository(full_name="naver/ngrinder"),
 Repository(full_name="spring-projects/spring-data-neo4j"),
 Repository(full_name="assertj/assertj-core"),
 Repository(full_name="FasterXML/jackson-databind"),
 Repository(full_name="robovm/robovm"),
 Repository(full_name="JSQLParser/JSqlParser"),
 Repository(full_name="mcMMO-Dev/mcMMO"),
 Repository(full_name="JodaOrg/joda-time"),
 Repository(full_name="datastax/java-driver"),
 Repository(full_name="java-native-access/jna"),
 Repository(full_name="spring-projects/spring-data-jpa"),
 Repository(full_name="jacoco/jacoco"),
 Repository(full_n

In [ ]:
count = 0

for project in sampling:
    project_name = project.full_name.split("/")[1]
    
    
    if not os.path.isfile('%s/%s-config.json'%(CONFIG_FILES_PATH, project_name)):
        count+=1
        print(project.full_name)
        config={
            'project': project_name,
            'git_url': project.clone_url,
            'last_commit': project.get_commits()[0].sha[0:8],
            'experiment': 1
        }

    with open('%s/%s-config.json'%(CONFIG_FILES_PATH, project_name), 'w') as outfile:
        json.dump(config, outfile, indent=4)
        
    if count == 40: break

    

In [ ]:
for f in os.listdir(CONFIG_FILES_PATH):
    path = os.path.join(CONFIG_FILES_PATH, f)
    if os.path.isfile(path):
        with open(path, 'r') as f:
            config_file = json.load(f)
            
            project_name = config_file['project']
            folder  = "/home/projects/%s" % project_name
            git_url = config_file['git_url']

            # CHECK IF PROJECT EXISTS
            if os.path.exists(folder):
                print(" -> Project exist in local folder!")
            else:
                print(" -> Project %s cloned!"%project_name)
                !git clone $git_url $folder > /dev/null 2>&1
            